In [ ]:
# started 10-12-2025
# https://hub.worldpop.org/geodata/listing?id=77 

In [5]:
import pandas as pd

import os
import zipfile
import pandas as pd
import numpy as np

In [3]:
fires = pd.read_csv("../attempt 3/fires_with_ndvi.csv") 

In [4]:
fires.head()

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,OBJECTID.1,temp_max_F,humidity_pct,precip_in,windspeed_mph,time,ndvi
0,110,2005,2453540.5,10.0,Equipment Use,33.718889,-117.433611,110,73.04,89,0.062992,6.028589,1970-01-01 00:00:00.002453540,5016.0
1,155,2005,2453411.5,3.0,Debris Burning,34.748333,-119.410278,155,58.46,79,0.000000,4.536979,1970-01-01 00:00:00.002453411,3357.0
2,178,2005,2453544.5,4.2,Equipment Use,34.466667,-119.828333,178,77.54,76,0.000000,15.972654,1970-01-01 00:00:00.002453544,4356.0
3,1053,2005,2453559.5,3.0,Miscellaneous,34.479444,-118.768611,1053,82.76,86,0.000000,6.028589,1970-01-01 00:00:00.002453559,3124.0
4,1282,2005,2453582.5,2.0,Lightning,33.110833,-116.847222,1282,87.62,88,0.000000,8.763207,1970-01-01 00:00:00.002453582,4180.0


In [8]:
fire_years = fires['FIRE_YEAR'].unique()

In [10]:
pop_folder = os.path.join("..", "pop")

In [11]:
fire_years, pop_folder

(array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]),
 '..\\pop')

In [19]:
def get_pop_densities_for_year(year, fires_subset, pop_folder, chunksize=100000):
    """
    For a given year, look up the nearest population density for each fire.
    
    Parameters
    ----------
    year : int
        Year of the fires.
    fires_subset : pd.DataFrame
        Subset of the fires dataframe with LATITUDE, LONGITUDE for that year.
    pop_folder : str
        Path to the pop folder containing yearly zip files.
    chunksize : int
        Number of rows to read at once from the population CSV.
    
    Returns
    -------
    list
        Population densities corresponding to fires_subset (same order).
    """
    # Path to outer zip
    outer_zip = os.path.join(pop_folder, f"50_US_states_1km_{year}_PD_UNadj.zip")
    
    with zipfile.ZipFile(outer_zip, 'r') as z1:
        # Find the nested zip file inside
        nested_name = [n for n in z1.namelist() if n.endswith(".zip")][0]
        
        with z1.open(nested_name) as nested_bytes:
            with zipfile.ZipFile(nested_bytes, 'r') as z2:
                # Find the CSV inside
                csv_name = [n for n in z2.namelist() if n.endswith(".csv")][0]
                
                # Open the CSV in chunks (comma-separated, header in first row)
                with z2.open(csv_name) as csv_file:
                    reader = pd.read_csv(csv_file, chunksize=chunksize)
                    
                    # Initialize best distances and densities
                    coords = fires_subset[["LONGITUDE","LATITUDE"]].to_numpy()
                    best_dists = np.full(len(coords), np.inf)
                    best_z = np.full(len(coords), np.nan)
                    
                    # Go chunk by chunk
                    for chunk in reader:
                        # Ensure numeric types
                        pop_coords = chunk[["X","Y"]].astype(float).to_numpy()
                        pop_z = chunk["Z"].astype(float).to_numpy()
                        
                        # Compute distances for each fire
                        for i, (lon, lat) in enumerate(coords):
                            dists = np.sqrt((pop_coords[:,0] - lon)**2 + (pop_coords[:,1] - lat)**2)
                            min_idx = np.argmin(dists)
                            if dists[min_idx] < best_dists[i]:
                                best_dists[i] = dists[min_idx]
                                best_z[i] = pop_z[min_idx]
                    
                    return best_z.tolist()


In [20]:
pop_density_list = np.full(len(fires), np.nan)

# Process year by year
for year in fire_years:
    # Get indices of fires in this year
    idx = fires.index[fires['FIRE_YEAR'] == year]
    fires_subset = fires.loc[idx]
    
    # Compute nearest population densities for this subset
    densities = get_pop_densities_for_year(year, fires_subset, pop_folder)
    
    # Assign densities to the corresponding positions
    pop_density_list[idx] = densities

# Add as a new column
fires['pop_density'] = pop_density_list

# Show first few rows to verify
fires.head()

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,OBJECTID.1,temp_max_F,humidity_pct,precip_in,windspeed_mph,time,ndvi,pop_density
0,110,2005,2453540.5,10.0,Equipment Use,33.718889,-117.433611,110,73.04,89,0.062992,6.028589,1970-01-01 00:00:00.002453540,5016.0,1631.960938
1,155,2005,2453411.5,3.0,Debris Burning,34.748333,-119.410278,155,58.46,79,0.000000,4.536979,1970-01-01 00:00:00.002453411,3357.0,0.696928
2,178,2005,2453544.5,4.2,Equipment Use,34.466667,-119.828333,178,77.54,76,0.000000,15.972654,1970-01-01 00:00:00.002453544,4356.0,24.228647
3,1053,2005,2453559.5,3.0,Miscellaneous,34.479444,-118.768611,1053,82.76,86,0.000000,6.028589,1970-01-01 00:00:00.002453559,3124.0,0.044615
4,1282,2005,2453582.5,2.0,Lightning,33.110833,-116.847222,1282,87.62,88,0.000000,8.763207,1970-01-01 00:00:00.002453582,4180.0,0.154729


In [ ]:
#fires.to_csv("fires_with_pop.csv", index=False)


NameError: name 'fires' is not defined